# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL."}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to identify trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News,

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Calendar Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Mate

Thought: I need to gather insights from the Data Analyst Agent's findings on AAPL and consider the user-defined risk tolerance and trading preferences to develop trading strategies that are profitable and align with the preferences.

Action: Search the internet
Action Input: {"search_query": "Data Analyst insights on AAPL"} 


Search results: Title: Apple Inc. Common Stock (AAPL) Analyst Research - Nasdaq
Link: https://www.nasdaq.com/market-activity/stocks/aapl/analyst-research
Snippet: Based on analysts offering 12 month price targets for AAPL in the last 3 months. The average price target is $204.77 with a high estimate of $275 and a low ...
---
Title: Apple Inc. (AAPL) Analyst Ratings, Estimates & Forecasts
Link: https://finance.yahoo.com/quote/AAPL/analysis/
Snippet: See Apple Inc. (AAPL) stock analyst estimates, including earnings and revenue, EPS, upgrades and downgrades.
---
Title: Exploring Predictive Insights in AAPL Stock Prices Using Machine ...
Link: https://medium.com/inst

I need to ensure that the team fully understands the approved trading strategies for AAPL and the current market conditions to determine the best execution methods.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop detailed execution plans for Momentum, Swing, Options, Sector Rotation, and Risk Management trading strategies for AAPL based on current market conditions and user preferences.", "context": "The user has a medium risk tolerance and prefers day trading. The Data Analyst has provided insights on market trends and potential strategies."}

> Entering new CrewAgentExecutor chain...
I need to gather detailed information on Momentum, Swing, Options, Sector Rotation, and Risk Management trading strategies specifically for AAPL in the current market conditions based on the user's preferences.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/trading/momentum-trading-strategies/"} 

Pag

Final Answer: Unfortunately, I was unable to retrieve detailed insights on approved trading strategies for AAPL from the provided website. I will continue to gather information from other sources to help develop execution plans for the Trading Strategy Developer.

> Finished chain.
 

Unfortunately, I was unable to retrieve detailed insights on approved trading strategies for AAPL from the provided website. I will continue to gather information from other sources to help develop execution plans for the Trading Strategy Developer.

Final Answer:
Based on the insights provided by the Data Analyst and considering the user-defined risk tolerance (Medium) and trading preferences (Day Trading), the potential trading strategies for AAPL are as follows:

1. Momentum Trading Strategy Execution Plan:
- Buy AAPL stocks when the price is trending upwards with a clear uptrend confirmed by technical indicators such as moving averages and MACD.
- Set a stop-loss order below the recent swing low to pr

Final Answer: Page Not Found

> Finished chain.
 

Page Not Found

Thought: The tool "Delegate work to co-worker" did not provide the expected result. I need to ask the Risk Advisor directly for the risk analysis and mitigation recommendations for the proposed trading strategies for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Risk Advisor", "question": "Please evaluate the risks associated with the proposed trading strategies and execution plans for AAPL, and provide detailed analysis of potential risks and suggest mitigation strategies.", "context": "Based on the insights provided by the Data Analyst and considering the user-defined risk tolerance (Medium) and trading preferences (Day Trading)"}

> Entering new CrewAgentExecutor chain...
I need to gather information on the risks associated with the proposed trading strategies for AAPL in order to provide a detailed analysis and suggest mitigation strategies that align with the firm's risk tolerance.

Action: S

Thought: The Risk Advisor has provided valuable resources related to risk factors and mitigation strategies for AAPL. I need to review these sources and compile a comprehensive risk analysis report.

Final Answer:
After reviewing the provided resources related to risk factors and mitigation strategies for AAPL, I have compiled a comprehensive risk analysis report detailing potential risks and mitigation recommendations for the proposed trading strategies.

Risk Analysis Report for AAPL Trading Strategies:

1. Legal & Regulatory Risks:
- Apple (AAPL) stock faces legal and regulatory risks that could impact its share price.
- Mitigation Strategy: Stay updated on legal developments and regulatory changes affecting AAPL to make informed trading decisions.

2. Market Risks:
- The stock market is subject to volatility and external factors that can impact AAPL's performance.
- Mitigation Strategy: Implement risk management techniques such as setting stop-loss orders and diversifying the portf

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

After reviewing the provided resources related to risk factors and mitigation strategies for AAPL, I have compiled a comprehensive risk analysis report detailing potential risks and mitigation recommendations for the proposed trading strategies.

Risk Analysis Report for AAPL Trading Strategies:

1. Legal & Regulatory Risks:
- Apple (AAPL) stock faces legal and regulatory risks that could impact its share price.
- Mitigation Strategy: Stay updated on legal developments and regulatory changes affecting AAPL to make informed trading decisions.

2. Market Risks:
- The stock market is subject to volatility and external factors that can impact AAPL's performance.
- Mitigation Strategy: Implement risk management techniques such as setting stop-loss orders and diversifying the portfolio to mitigate market risks.

3. Competition Risks:
- Apple faces competition from other tech companies that could affect its market share and profitability.
- Mitigation Strategy: Monitor competitor performance and industry trends to adjust trading strategies accordingly.

4. Technology Risks:
- Rapid technological advancements can impact AAPL's product offerings and consumer demand.
- Mitigation Strategy: Stay informed about emerging technologies and their potential impact on AAPL's business to make strategic trading decisions.

5. Economic Risks:
- Economic factors such as inflation, interest rates, and global economic conditions can influence AAPL's stock price.
- Mitigation Strategy: Monitor economic indicators and market trends to anticipate potential risks and adjust trading strategies accordingly.

6. Financial Risks:
- Apple's financial performance and management decisions can impact its stock price and investor confidence.
- Mitigation Strategy: Conduct thorough financial analysis and due diligence to assess AAPL's financial health and make informed trading decisions.

Overall, a proactive approach to risk management and continuous monitoring of market conditions are essential to successfully navigate the risks associated with trading AAPL stocks. By implementing sound risk mitigation strategies and staying informed about potential risks, traders can optimize their trading performance and protect their capital effectively.